In [12]:
import pandas as pd
import numpy as np
import json
import requests
from pprint import pprint
# from config import api_key
from config import gkey
import gmaps
gmaps.configure(api_key=gkey)

In [13]:
weather_path_1 = "output_weatherdata/weather_data_sunrise.csv"
sunrise_weather_df = pd.read_csv(weather_path_1, delimiter = ',')
sunrise_weather_df

weather_path_2 = "output_weatherdata/weather_data_solar_noon.csv"
solar_noon_weather_df = pd.read_csv(weather_path_2, delimiter = ',')
solar_noon_weather_df

weather_path_3 = "output_weatherdata/weather_data_sunset.csv"
sunset_weather_df = pd.read_csv(weather_path_3, delimiter = ',')
sunset_weather_df

,city id,city,country,lat,lon,temp_sunset,hum_sunset,cloudiness_sunset,windiness_sunset
0,1493756,Razdolinsk,RU,58.4211,94.6658,-16.29,89,100,2.01
1,2523116,Siculiana,IT,37.3344,13.4216,52.21,88,1,1.99
2,1489423,Tomskaya,RU,58.7844,64.1901,-15.52,88,99,4.56
3,5976257,Holman,CA,70.7342,-117.7546,-15.21,89,100,11.21
4,5865670,Kaktovik,US,70.1319,-143.6239,-9.40,84,90,31.07
...,...,...,...,...,...,...,...,...,...
995,1304138,Ninglum,MM,25.2167,97.2333,79.83,32,0,0.20
996,3315550,Emlagh,IE,53.5542,-10.1344,45.93,82,0,21.00
997,2078024,Adelaide River,AU,-13.2379,131.1056,79.61,91,100,3.78
998,5866726,Kotzebue,US,66.8983,-162.5967,-10.08,70,1,5.75


In [14]:
weather_merge = pd.merge(sunrise_weather_df,solar_noon_weather_df,on='city id')
weather_merge = pd.merge(weather_merge,sunset_weather_df,on='city id')

city_IDs = weather_merge["city id"].unique()
coordinates = []
for ID in city_IDs:
    x = weather_merge.loc[weather_merge["city id"] == ID,"lat"].item()
    y = weather_merge.loc[weather_merge["city id"] == ID,"lon"].item()
    coordinates.append((x,y))
print(coordinates) 
weather_merge.head()

[(58.4211, 94.6658), (37.3344, 13.4216), (58.7844, 64.1901), (70.7342, -117.7546), (70.1319, -143.6239), (54.8002, -66.8318), (-32.7075, 26.2956), (15.25, -61.3833), (57.1281, 73.8282), (-18.7633, -50.9417), (48.9, -72.6656), (17.0432, 99.0813), (-27.8333, -70.1), (58.4547, -78.1014), (17.6167, 80.0167), (-34.0, -60.0), (47.1667, 110.8167), (70.7333, -52.65), (-40.7333, -65.0333), (-1.3518, 127.6693), (-1.8333, 109.9667), (-21.8333, 46.9333), (36.6, 94.1), (62.891999999999996, 54.9635), (-40.9, 176.2167), (50.0833, 132.1667), (40.9333, 73.0), (9.0833, 105.0833), (32.9312, 12.082), (-2.4431, -54.7083), (-25.0269, 33.0989), (-14.0, -65.5), (54.4933, 99.0021), (47.9714, 80.4392), (53.5501, -117.2356), (52.1453, 93.9192), (51.9678, -8.9825), (45.55, 116.8333), (64.3201, -96.0212), (64.9308, -125.5003), (49.001999999999995, -111.963), (-20.7333, 139.5), (62.9996, -135.004), (-3.7802, 126.5171), (29.05, 93.2), (13.0942, 122.96), (10.0, 6.0), (-21.8667, 166.05), (60.3964, -1.3530000000000002)

,city id,city_x,country_x,lat_x,lon_x,temp_sunrise,hum_sunrise,cloudiness_sunrise,windiness_sunrise,city_y,...,cloudiness_solar_noon,windiness_solar_noon,city,country,lat,lon,temp_sunset,hum_sunset,cloudiness_sunset,windiness_sunset
0,1493756,Razdolinsk,RU,58.4211,94.6658,-24.74,87,76,2.98,Razdolinsk,...,62,5.35,Razdolinsk,RU,58.4211,94.6658,-16.29,89,100,2.01
1,2523116,Siculiana,IT,37.3344,13.4216,46.29,92,0,1.01,Siculiana,...,0,8.01,Siculiana,IT,37.3344,13.4216,52.21,88,1,1.99
2,1489423,Tomskaya,RU,58.7844,64.1901,-13.68,89,99,5.75,Tomskaya,...,99,6.24,Tomskaya,RU,58.7844,64.1901,-15.52,88,99,4.56
3,5976257,Holman,CA,70.7342,-117.7546,-22.00,76,75,17.27,Holman,...,75,17.27,Holman,CA,70.7342,-117.7546,-15.21,89,100,11.21
4,5865670,Kaktovik,US,70.1319,-143.6239,-11.20,77,90,33.38,Kaktovik,...,90,37.98,Kaktovik,US,70.1319,-143.6239,-9.40,84,90,31.07


In [15]:
#Plot Heatmap
fig = gmaps.figure()


humidity = weather_merge.loc[:,"hum_solar_noon"].values

#create heat layer
heat_layer = gmaps.heatmap_layer(coordinates,weights=humidity,dissipating=False,max_intensity=10,point_radius=1)

#Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
df = weather_merge.loc[(weather_merge["temp_sunrise"] >= 60) & (weather_merge["temp_solar_noon"] <= 90) & (weather_merge["temp_sunset"] >= 65) & (weather_merge["cloudiness_sunset"] == 0),:]
df = df.loc[:,["city id","city_x","country_x","lat_x","lon_x"]].rename(columns={"city_x": "city", "country_x": "country", "lat_x": "latitude", "lon_x": "longitude"})
df["name"] = ""
# df["price_level"] = ""
df["rating"] = ""

df

,city id,city,country,latitude,longitude,name,rating
68,3446232,Tramandaí,BR,-29.9847,-50.1336,,
71,2380519,Chingueṭṭi,MR,20.4404,-12.2622,,
80,1182998,Bela,PK,26.2271,66.3111,,
104,2250677,Kayar,SN,14.9192,-17.1211,,
106,288602,Camp,OM,20.6667,58.8833,,
113,3855715,Eugenio Bustos,AR,-33.7773,-69.0704,,
123,286245,Sur,OM,22.5667,59.5289,,
126,2356982,Province de l’Oudalan,BF,14.6667,-0.3333,,
182,3363441,Ocean View,ZA,-34.1486,18.3536,,
194,8030386,Safende,CV,15.3690,-25.8398,,


In [19]:
target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

params = {
    "key": gkey
}
# use iterrows to iterate through pandas dataframe
for index, row in df.iterrows():

    # get city and country from df
    city = row['city']
    country = row['country']
    print(f"{city},{country}")

    # add keyword to params dict
    params['query'] = f"best hotel in {city},{country}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(target_url, params=params).json()
    
    # extract results
    
    try:
          results = response['results'][0]
            
          print(f"Closest hotel is {results['name']}.")
        
          df.loc[index, 'name'] = results['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
          df.loc[index, 'rating'] = results['rating']
        
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")
        
    print("------------")

Tramandaí,BR
Retrieving Results for Index 68: Tramandaí.
Closest hotel is Pousada Morada da Plataforma.
------------
Chingueṭṭi,MR
Retrieving Results for Index 71: Chingueṭṭi.
Closest hotel is Beverly Wilshire, A Four Seasons Hotel.
------------
Bela,PK
Retrieving Results for Index 80: Bela.
Closest hotel is The Nines, a Luxury Collection Hotel, Portland.
------------
Kayar,SN
Retrieving Results for Index 104: Kayar.
Closest hotel is Fairmont Banff Springs.
------------
Camp,OM
Retrieving Results for Index 106: Camp.
Closest hotel is Hotel Eastlund.
------------
Eugenio Bustos,AR
Retrieving Results for Index 113: Eugenio Bustos.
Closest hotel is Posada La Celia.
------------
Sur,OM
Retrieving Results for Index 123: Sur.
Closest hotel is Sur Inn Hotel Apartments.
------------
Province de l’Oudalan,BF
Retrieving Results for Index 126: Province de l’Oudalan.
Closest hotel is Stephanie Inn & Dining Room.
------------
Ocean View,ZA
Retrieving Results for Index 182: Ocean View.
Closest hotel

In [20]:
df

,city id,city,country,latitude,longitude,name,rating
68,3446232,Tramandaí,BR,-29.9847,-50.1336,Pousada Morada da Plataforma,4.8
71,2380519,Chingueṭṭi,MR,20.4404,-12.2622,"Beverly Wilshire, A Four Seasons Hotel",4.7
80,1182998,Bela,PK,26.2271,66.3111,"The Nines, a Luxury Collection Hotel, Portland",4.4
104,2250677,Kayar,SN,14.9192,-17.1211,Fairmont Banff Springs,4.7
106,288602,Camp,OM,20.6667,58.8833,Hotel Eastlund,4.4
113,3855715,Eugenio Bustos,AR,-33.7773,-69.0704,Posada La Celia,4.8
123,286245,Sur,OM,22.5667,59.5289,Sur Inn Hotel Apartments,5
126,2356982,Province de l’Oudalan,BF,14.6667,-0.3333,Stephanie Inn & Dining Room,4.7
182,3363441,Ocean View,ZA,-34.1486,18.3536,Bethany Beach Ocean Suites Residence Inn by Ma...,4.4
194,8030386,Safende,CV,15.3690,-25.8398,Lafayette Park Hotel & Spa,4.5


In [21]:
df = df.loc[df['name'] != '',:]
df

,city id,city,country,latitude,longitude,name,rating
68,3446232,Tramandaí,BR,-29.9847,-50.1336,Pousada Morada da Plataforma,4.8
71,2380519,Chingueṭṭi,MR,20.4404,-12.2622,"Beverly Wilshire, A Four Seasons Hotel",4.7
80,1182998,Bela,PK,26.2271,66.3111,"The Nines, a Luxury Collection Hotel, Portland",4.4
104,2250677,Kayar,SN,14.9192,-17.1211,Fairmont Banff Springs,4.7
106,288602,Camp,OM,20.6667,58.8833,Hotel Eastlund,4.4
113,3855715,Eugenio Bustos,AR,-33.7773,-69.0704,Posada La Celia,4.8
123,286245,Sur,OM,22.5667,59.5289,Sur Inn Hotel Apartments,5
126,2356982,Province de l’Oudalan,BF,14.6667,-0.3333,Stephanie Inn & Dining Room,4.7
182,3363441,Ocean View,ZA,-34.1486,18.3536,Bethany Beach Ocean Suites Residence Inn by Ma...,4.4
194,8030386,Safende,CV,15.3690,-25.8398,Lafayette Park Hotel & Spa,4.5


In [22]:
city_IDs = df["city id"].unique()
coordinates = []
for ID in city_IDs:
    x = df.loc[df["city id"] == ID,"latitude"].item()
    y = df.loc[df["city id"] == ID,"longitude"].item()
    coordinates.append((x,y))
print(coordinates) 

[(-29.9847, -50.1336), (20.4404, -12.2622), (26.2271, 66.3111), (14.9192, -17.1211), (20.6667, 58.8833), (-33.7773, -69.0704), (22.5667, 59.5289), (14.6667, -0.3333), (-34.1486, 18.3536), (15.369000000000002, -25.8398), (15.032, 45.8283), (21.05, 79.0333), (-37.8791, -67.7956), (-2.2, -81.0), (-28.6333, 122.4167), (-32.0333, -52.0917), (4.7485, -6.6363), (-31.4667, 119.4667), (18.0206, 101.8983), (-33.9167, 18.9167), (29.0415, 67.8239), (-30.0167, -70.5333), (-30.95, 121.1667), (-28.3942, 121.2596), (12.3807, 17.0568), (-22.0833, 15.3333), (21.0964, 77.0586), (16.2717, -0.0447), (16.7735, -3.0074), (-37.0167, 175.85), (-30.0717, 114.9672), (14.3636, 48.9797), (-27.8667, 151.2667)]


In [23]:
#Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
#Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
hotel_df = df[['latitude', 'longitude']]

hotel_layer = gmaps.symbol_layer(
    hotel_df, fill_color="green", stroke_color="green", scale=2
)
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))